## Explorando e modelando os dados

In [41]:
import pandas as pd
df = pd.read_json('https://gist.githubusercontent.com/386er/84a78c9dd226a9395818/raw/dbed7a575d899876bff063a3590081f40816309e/airports.json')

df.head()

,City,DBTZ,Name,Country,IATA/FAA,Longitude,ICAO,Airport ID,Latitude,Timezone,DST,destinations
0,Postville,A,Postville Airport,Canada,YSO,-59.785278,CCD4,7252,54.910278,223.0,-4.0,"[5492, 188, 5502]"
1,Osubi,U,Warri Airport,Nigeria,QRW,5.450000,DNSU,6972,5.31,50.0,1.0,"[260, 273]"
2,Rockland,A,Knox County Regional Airport,United States,RKD,-69.099230,KRKD,4026,44.0601111,56.0,-5.0,[3448]
3,Jacksn Hole,A,Jackson Hole Airport,United States,JAC,-110.737750,KJAC,4027,43.607333333,6451.0,-7.0,"[3536, 3751]"
4,Flores,U,Mundo Maya International,Guatemala,FRS,-89.866383,MGTK,4024,16.913819,427.0,-6.0,"[1957, 1767]"


In [42]:
# corrigindo falta de latitude nos aeroportos [5562, 5674, 5675], fonte: latitude.to

# df.loc[df['Airport ID'] == 5674]
df.loc[df['Airport ID'] == 5674, ['Latitude']] = '14.9211829819'
df.loc[df['Airport ID'] == 5674, ['Longitude']] = '-23.4896713746'

# Robin Hood Airport Doncaster Sheffield
df.loc[df['Airport ID'] == 5562, ['Latitude']] = '53.471498114'
df.loc[df['Airport ID'] == 5562, ['Longitude']] = '-1.002666656'

# Sao Filipe Airport
df.loc[df['Airport ID'] == 5675, ['Latitude']] = '14.8764631608'
df.loc[df['Airport ID'] == 5675, ['Longitude']] = '-24.474664768'

df.head()

,City,DBTZ,Name,Country,IATA/FAA,Longitude,ICAO,Airport ID,Latitude,Timezone,DST,destinations
0,Postville,A,Postville Airport,Canada,YSO,-59.7853,CCD4,7252,54.910278,223.0,-4.0,"[5492, 188, 5502]"
1,Osubi,U,Warri Airport,Nigeria,QRW,5.45,DNSU,6972,5.31,50.0,1.0,"[260, 273]"
2,Rockland,A,Knox County Regional Airport,United States,RKD,-69.0992,KRKD,4026,44.0601111,56.0,-5.0,[3448]
3,Jacksn Hole,A,Jackson Hole Airport,United States,JAC,-110.738,KJAC,4027,43.607333333,6451.0,-7.0,"[3536, 3751]"
4,Flores,U,Mundo Maya International,Guatemala,FRS,-89.8664,MGTK,4024,16.913819,427.0,-6.0,"[1957, 1767]"


In [43]:
print(f'Numero de aeroportos na base de dados: {len(df)}\n')
print(df.dtypes)

Numero de aeroportos na base de dados: 3266

City             object
DBTZ             object
Name             object
Country          object
IATA/FAA         object
Longitude        object
ICAO             object
Airport ID        int64
Latitude         object
Timezone        float64
DST             float64
destinations     object
dtype: object


O Grafo será representado por um dict(vertice:dict(vertice_adj:distancia)

*nó/vertice = Aeroporto


O que precisa ser feito: 

    Separar dados úteis
    Nome - ID - Lat/Long - Adjacentes - ICAO
    *Apesar do IATA ser mais popular, o ICAO fornece ids únicos

    Dict ID:Nome pra recuperar depois e exibir o caminho pelos nomes dos aeroportos

    Pegar o ID do nó e calcular distancias dos adjacentes, construindo assim um dicionário 
    ID:Distancia que será o 'valor' do dicionário vertice:adjacentes, nosso grafo

    Farei isso transformando o dataframe pra dict e iterando sobre o dicionario calculando as 
    distancias pro adjacente e trocando o campo 'detinations' por esse novo dicionario: 
    {idDestino:distancia}


In [44]:
# Retirando colunas desnecessarias para nossa aplicacao
df.drop(columns=['Country', 'Timezone', 'DBTZ', 'IATA/FAA', 'City', 'DST'], inplace = True)
df.head()

,Name,Longitude,ICAO,Airport ID,Latitude,destinations
0,Postville Airport,-59.7853,CCD4,7252,54.910278,"[5492, 188, 5502]"
1,Warri Airport,5.45,DNSU,6972,5.31,"[260, 273]"
2,Knox County Regional Airport,-69.0992,KRKD,4026,44.0601111,[3448]
3,Jackson Hole Airport,-110.738,KJAC,4027,43.607333333,"[3536, 3751]"
4,Mundo Maya International,-89.8664,MGTK,4024,16.913819,"[1957, 1767]"


## Funções da nossa aplicação

In [52]:
from geopy import distance

# funcao que recebe coordenadas para calcular a distancia geodesia entre dois nos adjacentes
def calc_distance(origin_lat, origin_long, dest_lat, dest_long):
	origin = (origin_lat, origin_long)
	destination = (dest_lat, dest_long)

	dis = distance.distance(origin, destination).km
	return dis
  
# funcao que vai iterar por todo dic de aeroportos a fim de calcular distancias pros adjacentes e substituir {destinations: {airportID: distance}}
def set_distances(dic):
	for key in dic:
		origin = dic.get(key)
		if(origin):
			origin_lat = origin['Latitude']
			origin_long = origin['Longitude']
			destinations = origin['destinations']
			dest_dist = {}

		for dest in destinations:
			dest_entry = dic.get(int(dest))
			if(dest_entry):
				dest_lat = dest_entry['Latitude']
				dest_long = dest_entry['Longitude']
				distance = calc_distance(origin_lat, origin_long, dest_lat, dest_long)
				dest_dist[int(dest)] = distance
		
		dic[key]['destinations'] = dest_dist

# Funcao que isola dicionario para se adequar ao modelo do grafo que precisamos {airportID: {destID1: distance, destID2: distance, ...}}
def get_graph(dic):
	graph = {}
	for key in dic:
		graph[key] = dic[key]['destinations']
	return graph

# Classe que representa uma heap binaria de minimo, necessaria ao nosso algoritmo Dijkstra  
class BinHeap:
	def __init__(self):
		self.heap_list = [(0,)]
		self.heap_size = 0

	def size(self):
		return self.heap_size

	def percolate_up(self, i):
		while i//2 > 0:
			if self.heap_list[i] < self.heap_list[i//2]:
				temp = self.heap_list[i//2]
				self.heap_list[i//2] = self.heap_list[i]
				self.heap_list[i] = temp
			i = i//2

	def insert(self, k):
		self.heap_list.append(k)
		self.heap_size += 1
		self.percolate_up(self.heap_size)

	def percolate_down(self, i):
		while(i*2) <= self.heap_size:
			min_child = self.min_child(i)
			if self.heap_list[i] > self.heap_list[min_child]:
				temp = self.heap_list[i]
				self.heap_list[i] = self.heap_list[min_child]
				self.heap_list[min_child] = temp
			i = min_child

	def min_child(self, i):
		if (i*2+1) > self.heap_size:
			return i*2
		else:
			if self.heap_list[i*2][0] < self.heap_list[i*2+1][0]:
				return i*2
			else:
				return i*2+1

	def pop(self):
		r = self.heap_list[1]
		self.heap_list[1] = self.heap_list[self.heap_size]
		self.heap_size -= 1
		self.heap_list.pop()
		self.percolate_down(1)
		return r

	def build_heap(self, list):
		i = len(list) // 2
		self.heap_size = len(list)
		self.heap_list = [0] + list[:]
		while(i > 0):
			self.percolate_down(i)
			i -= 1

# Funcao que implementa o algoritmo Dijkstra, retornando as menores distancias de um vertice a todos os outros do grafo, e armazenando os caminhos a percorrer recursivamente
def dijkstra(graph, starting_vertex):
	dist = {v: float('infinity') for v in graph}
	dist[starting_vertex] = 0
	pq = BinHeap()
	parent = {key: -1 for key in graph}

	pq.insert((0, starting_vertex))
	while pq.size() > 0:
		cur_dist, cur_v = pq.pop()

		# nos podem ser adicionados varias vezes a pq
		if cur_dist > dist[cur_v]:
			continue

		for adj, w in graph[cur_v].items():
			d = cur_dist + w
		# so atualiza o caminho se for menor que o ja encontrado anteriormente
			if d < dist[adj]:
				dist[adj] = d
				pq.insert((d, adj))
				parent[adj] = cur_v

	return dist, parent

# Funcao que imprime o caminho entre origem e destino a partir de um dic parents{} que armazena o vizinho mais próximo de cada vertice no caminho
def get_path(origem, destino, parents):    
	cur_node = destino
	caminho_reverso = [destino]
	caminho = []
	while cur_node != origem:
			caminho_reverso.append(parents[cur_node])
			cur_node = parents[cur_node]
	for i in range(len(caminho_reverso)): # Revertendo o caminho pra ordem correta
			caminho.append(caminho_reverso[-i-1])
	return caminho

# Funcao fachada da aplicacao, recebe o df de onde extrai o grafo, id do aeroporto de origem e uma lista de ids de aeroportos destino para os quais queremos calcular o menor caminho a partir da origem
def fachada(df, origem, destinos):
	dic = df.set_index('Airport ID').T.to_dict() # Saida: {airportID: {Name: 'name', destinations: {airportID: distance}, ...}}
	set_distances(dic)
	graph = get_graph(dic)
	
	sh_distances, sh_paths = dijkstra(graph, origem)
 
	for destino in destinos:
		caminho = get_path(origem, destino, sh_paths)
		print(f'A distancia de {dic[origem]["Name"]} para {dic[destino]["Name"]} é de: {sh_distances[destino]} quilometros\n')
		print(f'O caminho a ser percorrido passa pelos seguintes aeroportos, em ordem:\n')
		for node in caminho:
			print(f'{dic[node]["Name"]}, {dic[node]["ICAO"]}')
		print('----------------------------\n')

## Aplicacao (usuario)

Esta aplicação tem como objetivo traçar as menores rotas entre um aeroporto de origem e uma lista de aerportos de destino a partir da origem. 

fachada(dataframe, origem, lista_de_destinos) é nossa função principal.
Ela pode ser alterada de acordo com a necessidade para diferentes utilizações.

Uma lista de airportID: Name está disponível num .txt anexado a este material, para fins de consulta.

In [60]:
fachada(df, 6972, [4026, 4027])

A distancia de Warri Airport para Knox County Regional Airport é de: 9064.393253812676 quilometros

O caminho a ser percorrido passa pelos seguintes aeroportos, em ordem:

Warri Airport, DNSU
Murtala Muhammed, DNMM
Leopold Sedar Senghor Intl, GOOY
Praia International Airport, RAI
General Edward Lawrence Logan Intl, KBOS
Knox County Regional Airport, KRKD
----------------------------

A distancia de Warri Airport para Jackson Hole Airport é de: 11993.514801159097 quilometros

O caminho a ser percorrido passa pelos seguintes aeroportos, em ordem:

Warri Airport, DNSU
Murtala Muhammed, DNMM
John F Kennedy Intl, KJFK
Denver Intl, KDEN
Jackson Hole Airport, KJAC
----------------------------



## * Criando lista de airport_ids

In [59]:
# Criando lista de airport_ids (rodar apenas 1x)
dicio = df.set_index('Airport ID').T.to_dict()
f = open("airport_ids.txt", 'a')
for key in dicio:
  f.write(f"{key}: {dicio[key]['Name']}\n")
f.close()